In [12]:
import pandas as pd
import os
import random

In [29]:
partition_idx = 0
n = 500
df = pd.read_csv(
        f"./data/partition/partition_{partition_idx}.txt", sep="\t", header=None
    )
# Get a sub-dataset
idx = random.sample(range(0, len(df)), n)

data = df.iloc[idx]

In [30]:
data

,0
2769,authority says protesters not bright sparks
2026,witnesses questioned over head on crash
1518,victory aware of disciplinary issues allsopp
2753,no tennant creek wind farm guarantee
1776,howard denies govt ignoring renewable energy
...,...
1770,owen to retire at seasons end
630,govt on attack over budget leaks
983,wallabies rest stars ahead of cup
260,brazil dam desperate search for survivors
